In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

tqdm.pandas()

In [8]:
df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')

In [9]:
pd.set_option('display.max_columns', None)

In [10]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [6]:
# Split the data set and extract time series features
X = df.loc[:,~df.columns.isin(['veranst_segment','vg_inkasso'])]
y = df['vg_inkasso']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [8]:
display(df.head(5))

,imp_id,gj,import,mufo_referenz_n,barcode_nr,veranst_segment,rekla_jn,vg_datum_von,vg_ort,vg_raum,nutzfall,nutzfall_raum,musikleiter_name,kapelle_name,tarif_nr,tarif_bez,nutzfall_nr,vg_inkasso,inkasso_netto,inkasso_brutto,veranst_geschaeftszeichen,veranst_name,veranst_strasse,veranst_plz,veranst_ort,nutzliznehm_geschaeftszeichen,nutzliznehm_name,nutzliznehm_vorname,nutzliznehm_strasse,nutzliznehm_ort,location,band,promoter,vg_state,vg_raum_clean,vg_raum_wo_stopwords,vg_raum_keywords_x,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,vg_raum_keywords_y,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,vg_state_y,state_bavaria,state_thuringia,state_baden-wuerttemberg,state_rhineland-palatinate,state_brandenburg,state_berlin,state_lower saxony,state_schleswig-holstein,state_hamburg,state_hesse,state_saarland,state_north rhine-westphalia,state_saxony-anhalt,state_saxony,state_mecklenburg-western pomerania,state_bremen,band_y,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,24643,2016,ABRE U INKA 2016,None,5027961235,4,NONRekla,2014-02-08,LISBERG,TURNHALLE SV RW LISBERG,20140208.0,TURNHALLE SV RW LISBERG,STAERK REINHOLD,DAS BOEHMISCHE FEUER,892,U-K (MUSIKER),18293091,210.00,NaN,NaN,1035530900,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V,SCHINDHOLZWEG 17,96194,WALSDORF,1510423300,BAYERISCHER BLASMUSIKVERBAND,None,None,MUENCHEN,LISBERG TURNHALLE SV RW LISBERG,DAS BOEHMISCHE FEUER,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V 96194,BAVARIA,TURNHALLE SV RW LISBERG,turnhalle sv rw lisberg,"[turnhalle, sv, rw, lisberg]",2014,2,5,1,"[turnhalle, sv, rw, lisberg]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,BAVARIA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DAS BOEHMISCHE FEUER,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939272,24643,2016,ABRE U INKA 2016,None,5053180236,3,NONRekla,2014-02-22,WIPPERDORF,STAATLICHE GRUNDSCHULE WIPPERDORF,20140222.0,STAATLICHE GRUNDSCHULE WIPPERDORF,PATSCHKE RAINER,KYFFHAEUSER TANZ BAN,1087,U-V II. 1 (+II 2),17923031,85.06,NaN,NaN,1374109400,WIPPERDORFER CARNEVAL CLUB E.V,STRASSE DER EINHEIT 6,99752,WIPPERDORF,1374109400,WIPPERDORFER CARNEVAL CLUB,None,STRASSE DER EINHEIT 6,WIPPERDORF,WIPPERDORF STAATLICHE GRUNDSCHULE WIPPERDORF,KYFFHAEUSER TANZ BAN,WIPPERDORFER CARNEVAL CLUB E.V 99752,THURINGIA,STAATLICHE GRUNDSCHULE WIPPERDORF,staatliche grundschule wipperdorf,"[grundschule, wipperdorf]",2014,2,5,1,"[grundschule, wipperdorf]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,THURINGIA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KYFFHAEUSER TANZ BAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939279,24643,2016,ABRE U INKA 2016,None,5053345001,3,NONRekla,2014-02-18,DINGOLFING,BMW-WERK DINGOLFING,20140218.0,BMW-WERK DINGOLFING,JUNDMAYER,BERUFSFACHSCHULE,1087,U-V II. 1 (+II 2),17757433,90.20,NaN,NaN,2000026695,BAYERISCHE MOTORENWERKE AG,MENGKOFENER STR. 13,84130,DINGOLFING,2000026695,BAYERISCHE MOTORE

In [9]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states

317011
176456
16


Extract inkasso distribution with respect to the time of the event

In [33]:
def extract_features(features, target):
    # stat_var = ['band']
    stat_var = ['vg_state']
    # stat_var = ['promoter']
    # stat_var = ['band']
    df_stat = {}

    # 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
    # 2. Timestamp as index, and have location/band as columns in df
    # 3. Predefine the single df and iterate over it

    def get_descr_stat(row):
        # row represents each line(row) in the dataframe
        def descr_stat(row, var):
            # Initialize a empty pandas dataframe
            result = pd.Series(dtype='float64')

            if not pd.isnull(row[var]):
                #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
                matching_indexes = features[(features['vg_datum_von']<row['vg_datum_von'])&(features[var] == row[var])].index.values.tolist()

                inkasso =  list(target[target.index.isin(matching_indexes)].values)

                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso, dtype='float64').describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

            result.name = row.name
            return result

        descr_stat_result = pd.Series(dtype='float64')
        for v in stat_var:
            descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

        return descr_stat_result

 
    
    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

    # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
    df_featurize_descr_stat = features.progress_apply(get_descr_stat, axis=1)
    # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

    # display(df_featurize_descr_stat.head(5))
    return df_featurize_descr_stat

# # For training data - split the data and extract features
# # For testing data - Features can be extracted without splits

# # Splitting 1393792 entries to 128 splits, where each split contains 10889 entries
# X_train_splits = np.array_split(X_train,128)
# y_train_splits = np.array_split(y_train,128)

# df_featurize_descr_stat_result = pd.Series(dtype='float64')
# for i in range(0,len(X_train_splits)):
#     X_train_slice = X_train_splits[i]
#     y_train_slice = y_train_splits[i]
#     result = extract_features(X_train_slice, y_train_slice)
#     df_featurize_descr_stat_result = pd.concat([df_featurize_descr_stat_result, result])

# df_featurize_descr_stat_result.index.name = 'ID'
# df_featurize_descr_stat_result = df_featurize_descr_stat_result.drop(0, axis=1) # remove 0th index column with NaN values 
# display(df_featurize_descr_stat_result.head(5))

df_featurize_descr_stat = extract_features(X_train, y_train)

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

...

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max
ID,,,,,,,,,,,,,,,,,,,,,,,,
12151712,228.0,405.549649,1432.129394,11.02,17.440,18.24,21.800,22.80,35.9800,36.480,45.1000,54.720,67.650,72.50,85.9655,100.460,108.1635,135.242,157.0800,194.956,312.5975,444.854,1388.9360,12403.20
12885094,1629.0,1515.011056,21528.645329,1.10,16.110,18.44,18.640,22.80,23.3000,32.894,37.2800,45.680,54.720,67.44,76.6840,93.200,115.7600,138.944,184.4000,240.268,329.4100,539.366,1250.1480,689747.70
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.64,20.566,23.05,25.3025,36.520,43.0475,51.344,61.170,69.73,89.3980,109.920,147.5780,189.372,235.8075,303.324,398.8800,691.273,2687.6785,183608.57
10114364,256.0,667.039062,3472.382538,15.50,18.040,18.24,21.800,22.80,30.3300,36.480,45.6000,52.390,61.285,77.52,90.4250,114.000,149.2100,190.990,243.2375,319.200,400.9200,650.375,1445.3350,46779.55
13926216,2055.0,1192.751859,20982.932598,11.02,16.787,18.64,19.710,23.30,31.6450,37.504,45.6000,55.320,63.156,69.90,84.4330,97.386,122.0190,149.824,192.7000,244.650,307.6680,460.460,928.0400,565427.13


In [43]:
# Training set - Saving band feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [47]:
# Training set - Saving vg_state feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [32]:
# Training set - Saving promoter feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [49]:
# Testing set - Saving promoter feature inkasso distribution
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [34]:
# Testing set - Saving band feature inkasso distribution
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# Testing set - Saving vg_state feature inkasso distribution
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

Merge the extracted features into training set and testing set

In [11]:
# Merge the time series distribution of vg_inkasso values for training data of band, promoter, vg_state
data_path_train_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_train_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_train_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

if data_path_train_vg_state.exists() and data_path_train_band.exists() and data_path_train_promoter.exists():
    df_train_state = pd.read_parquet(data_path_train_vg_state)
    df_train_band = pd.read_parquet(data_path_train_band)
    df_train_promoter = pd.read_parquet(data_path_train_promoter)

    df_train_time_series_merged_descr_stat = df_train_state.merge(df_train_band, how='left', on='ID').merge(df_train_promoter,
                                                                                                    how='left',
                                                                                                        on='ID')

display(df_train_time_series_merged_descr_stat.head(5))
print(df_train_time_series_merged_descr_stat.shape)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12151712,228.0,405.549649,1432.129394,11.02,17.440,18.24,21.800,22.80,35.9800,36.480,45.1000,54.720,67.650,72.50,85.9655,100.460,108.1635,135.242,157.0800,194.956,312.5975,444.854,1388.9360,12403.20,7.0,124.758571,56.774754,78.82,78.8200,78.820,78.8200,78.820,78.8200,78.820,79.4970,81.528,83.5590,85.590,107.1630,128.736,150.3090,165.376,177.19,189.004,196.880,196.880,196.880,196.88,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12885094,1629.0,1515.011056,21528.645329,1.10,16.110,18.44,18.640,22.80,23.3000,32.894,37.2800,45.680,54.720,67.44,76.6840,93.200,115.7600,138.944,184.4000,240.268,329.4100,539.366,1250.1480,689747.70,4.0,124.642500,83.250302,36.48,41.6055,46.731,51.8565,56.982,62.1075,67.233,76.9035,95.664,114.4245,133.185,151.9455,170.706,189.4665,195.720,195.72,195.720,195.720,195.720,195.720,195.72,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.64,20.566,23.05,25.3025,36.520,43.0475,51.344,61.170,69.73,89.3980,109.920,147.5780,189.372,235.8075,303.324,398.8800,691.273,2687.6785,183608.57,21.0,19.247619,14.057616,15.59,15.7700,15.770,15.7700,15.770,15.7700,15.770,15.7700,15.770,15.7700,15.770,15.7700,15.770,15.7700,15.940,15.94,15.940,18.440,18.440,18.440,80.46,2.0,21.49,2.206173,19.93,20.086,20.242,20.398,20.554,20.71,20.866,21.022,21.178,21.334,21.49,21.646,21.802,21.958,22.114,22.27,22.426,22.582,22.738,22.894,23.05
10114364,256.0,667.039062,3472.382538,15.50,18.040,18.24,21.800,22.80,30.3300,36.480,45.6000,52.390,61.285,77.52,90.4250,114.000,149.2100,190.990,243.2375,319.200,400.9200,650.375,1445.3350,46779.55,172.0,2761.951163,3450.270464,155.04,616.7395,857.385,991.2335,1057.796,1099.1150,1203.985,1346.7000,1390.472,1452.4665,1537.640,1596.7530,1755.900,1902.2460,2094.460,2232.71,2501.264,3866.666,6552.499,12462.026,14307.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13926216,2055.0,1192.751859,20982.932598,11.02,16.787,18.64,19.710,23.30,31.6450,37.504,45.6000,55.320,63.156,69.90,84.4330,97.386,122.0190,149.824,192.7000,244.650,307.6680,460.460,928.0400,565427.13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(1393792, 72)


In [12]:
# Merge the time series distribution of vg_inkasso values for training data of band, promoter, vg_state
data_path_test_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_test_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_test_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

if data_path_test_vg_state.exists() and data_path_test_band.exists() and data_path_test_promoter.exists():
    df_test_state = pd.read_parquet(data_path_test_vg_state)
    df_test_band = pd.read_parquet(data_path_test_band)
    df_test_promoter = pd.read_parquet(data_path_test_promoter)

    df_test_time_series_merged_descr_stat = df_test_state.merge(df_test_band, how='left', on='ID').merge(df_test_promoter,
                                                                                                            how='left',
                                                                                                            on='ID')

display(df_test_time_series_merged_descr_stat.head(5))
print(df_test_time_series_merged_descr_stat.shape)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13331491,52410.0,789.620614,14192.979795,1.97,18.24,18.64,21.80,23.30,31.20,37.28,45.60,54.12,63.0600,69.90,82.40,95.060,117.75,141.300,182.40,230.82,306.510,451.810,863.9485,510768.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,1.0,78.25,78.250000,78.25,NaN
10619045,8343.0,421.600133,3431.393212,10.80,17.44,18.24,22.55,22.80,34.51,36.48,45.60,54.12,65.1990,70.80,84.56,91.200,114.00,135.300,162.36,205.20,255.360,354.120,588.8300,70573.56,1.0,36.0800,NaN,36.08,36.0800,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.08,36.080,36.080,36.080,36.080,36.0800,36.080,36.0800,36.080,36.0800,36.08,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,1.0,18.24,18.240000,18.24,NaN
11723390,41930.0,1208.057565,19775.388186,0.04,17.44,18.44,18.84,23.05,23.55,36.48,38.40,46.60,55.9200,68.47,79.54,95.526,120.00,149.120,193.60,248.57,331.920,497.880,1116.2200,689747.70,8.0,3693.8025,7388.111688,51.85,53.4285,55.007,58.775,75.680,92.585,107.946,119.447,130.948,138.141,139.59,141.039,141.660,141.660,141.660,2209.6375,5104.806,7999.9745,12040.441,16271.7905,20503.14,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,13430.2945,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,14.0,20503.14,19059.702143,295.01,5400.849921
10809092,31625.0,636.838359,11934.854355,1.97,18.24,18.64,22.80,23.10,33.55,37.28,46.10,54.72,63.8400,70.72,85.50,96.846,116.50,140.282,177.84,225.50,285.298,410.400,740.9560,510768.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.773,63.7995,66.826,69.8525,72.879,75.9055,78.932,81.9585,57.7465,84.985,88.0115,91.038,94.0645,97.091,100.1175,103.144,106.1705,109.197,112.2235,2.0,115.25,84.985000,54.72,42.801173
13349736,19746.0,854.077007,6619.089476,1.88,18.04,18.64,19.28,23.05,23.30,31.20,37.68,46.97,57.9525,70.65,91.10,114.000,144.32,184.400,237.50,316.92,466.155,886.455,2512.1675,224162.74,5.0,91.2360,51.904515,42.74,46.2860,49.832,53.378,56.924,60.470,62.896,65.322,67.748,70.174,72.60,79.278,85.956,92.634,99.312,105.9900,119.668,133.3460,147.024,160.7020,174.38,37.460,37.4600,37.460,37.4600,40.244,43.0280,45.812,48.5960,37.4600,51.380,58.3760,65.372,72.3680,79.364,86.3600,104.692,123.0240,141.356,159.6880,5.0,178.02,78.136000,37.46,59.307262


(348449, 72)


In [13]:
features = [
'place_kirche', 'place_hotel', 'place_cafe',
 'place_theater', 'place_club', 'place_halle',
 'place_gaststaette', 'place_festhalle', 'place_kulturzentrum',
 'place_festzelt', 'place_schloss', 'place_pub',
 'place_stadthalle', 'place_park', 'place_gasthof',
 'place_kabarett', 'place_arena', 'place_schlachthof',
 'place_wandelhalle', 'place_turnhalle', 'place_buergerhaus',
 'place_museum', 'place_rathaus', 'place_staatsbad',
 'place_zelt', 'place_jazz', 'place_forum',
 'place_gymnasium', 'place_schule', 'place_sporthalle', 

'band_kurorchester bad wil', 'band_musikverein harmonie', 'band_kasalla',
 'band_cat ballou', 'band_roncalli  royal orch', 'band_jugendblasorchester',
 'band_kurorchester bad pyr', 'band_hoehner', 'band_paveier',
 'band_domstuermer', 'band_kluengelkoepp', 'band_alleinunterhalter',
 'band_the gregorian voices', 'band_brings', 'band_musica hungarica',
 'band_concerto', 'band_bad salzuflen orches', 'band_musikverein stadtkap',
 'band_salonorchester hunga', 'band_miljoe', 'band_raeuber',
 'band_kabarett leipziger f', 'band_marita koellner', 'band_salon-orchester hung',
 'band_blaeck foeoess', 'band_schuelerinnen und sc', 'band_romain vicente',
 'band_staatliche kurkapell', 'band_musikzug der freiwil', 'band_funky marys',

'state_bavaria','state_rhineland-palatinate',
'state_baden-wuerttemberg',	'state_north rhine-westphalia',	
'state_thuringia','state_hesse',	
'state_brandenburg', 'state_schleswig-holstein',	
'state_berlin',	'state_mecklenburg-western pomerania',	
'state_lower saxony', 'state_hamburg',	
'state_saarland', 'state_saxony-anhalt',	
'state_saxony',	'state_bremen',

'vg_datum_year','vg_datum_month','vg_datum_day_of_week','vg_datum_season', 

'veranst_segment','vg_inkasso'
]

In [14]:
df_features = df[features]

In [15]:
# Train size = 1393792
# Test size = 348449
df_training = df_train_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(len(df_training))
display(df_training.head(5))

1393792


,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12151712,228.0,405.549649,1432.129394,11.02,17.440,18.24,21.800,22.80,35.9800,36.480,45.1000,54.720,67.650,72.50,85.9655,100.460,108.1635,135.242,157.0800,194.956,312.5975,444.854,1388.9360,12403.20,7.0,124.758571,56.774754,78.82,78.8200,78.820,78.8200,78.820,78.8200,78.820,79.4970,81.528,83.5590,85.590,107.1630,128.736,150.3090,165.376,177.19,189.004,196.880,196.880,196.880,196.88,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,8,4,3,4,191.52
12885094,1629.0,1515.011056,21528.645329,1.10,16.110,18.44,18.640,22.80,23.3000,32.894,37.2800,45.680,54.720,67.44,76.6840,93.200,115.7600,138.944,184.4000,240.268,329.4100,539.366,1250.1480,689747.70,4.0,124.642500,83.250302,36.48,41.6055,46.731,51.8565,56.982,62.1075,67.233,76.9035,95.664,114.4245,133.185,151.9455,170.706,189.4665,195.720,195.72,195.720,195.720,195.720,195.720,195.72,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,3,5,2,3,57.84
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.64,20.566,23.05,25.3025,36.520,43.0475,51.344,61.170,69.73,89.3980,109.920,147.5780,189.372,235.8075,303.324,398.8800,691.273,2687.6785,183608.57,21.0,19.247619,14.057616,15.5

In [33]:
df_testing = df_test_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(len(df_testing))

348449


In [34]:
display(df_testing.head(5))

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13331491,52410.0,789.620614,14192.979795,1.97,18.24,18.64,21.80,23.30,31.20,37.28,45.60,54.12,63.0600,69.90,82.40,95.060,117.75,141.300,182.40,230.82,306.510,451.810,863.9485,510768.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,1.0,78.25,78.250000,78.25,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,4,5,2,4,371.68
10619045,8343.0,421.600133,3431.393212,10.80,17.44,18.24,22.55,22.80,34.51,36.48,45.60,54.12,65.1990,70.80,84.56,91.200,114.00,135.300,162.36,205.20,255.360,354.120,588.8300,70573.56,1.0,36.0800,NaN,36.08,36.0800,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.08,36.080,36.080,36.080,36.080,36.0800,36.080,36.0800,36.080,36.0800,36.08,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,1.0,18.24,18.240000,18.24,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,9,3,4,2,18.24
11723390,41930.0,1208.057565,19775.388186,0.04,17.44,18.44,18.84,23.05,23.55,36.48,38.40,46.60,55.9200,68.47,79.54,95.526,120.00,149.120,193.60,248.57,331.920,497.880,1116.2200,689747.70,8.0,3693.8025,7388.111688,51.85,53.

In [38]:
test_copy = df_testing.copy()

In [39]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
test_copy.dropna(axis=0, thresh=152, inplace=True)
display(test_copy.head(5))
#(151287, 154)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10619045,8343.0,421.600133,3431.393212,10.80,17.44,18.24,22.55,22.80,34.51,36.48,45.600,54.120,65.1990,70.80,84.56,91.200,114.000,135.300,162.36,205.200,255.360,354.120,588.8300,70573.56,1.0,36.0800,NaN,36.08,36.0800,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.08,36.080,36.080,36.080,36.080,36.0800,36.080,36.0800,36.080,36.0800,36.08,18.240,18.240,18.240,18.24,18.240,18.240,18.240,18.240,18.2400,18.24,18.240,18.240,18.240,18.240,18.24,18.240,18.240,18.240,18.240,1.0,18.24,18.240000,18.24,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,9,3,4,2,18.24
11723390,41930.0,1208.057565,19775.388186,0.04,17.44,18.44,18.84,23.05,23.55,36.48,38.400,46.600,55.9200,68.47,79.54,95.526,120.000,149.120,193.60,248.570,331.920,497.880,1116.2200,689747.70,8.0,3693.8025,7388.111688,51.85,53.4285,55.007,58.775,75.680,92.585,107.946,119.447,130.948,138.141,139.59,141.039,141.660,141.660,141.660,2209.6375,5104.806,7999.9745,12040.441,16271.7905,20503.14,20503.140,20503.140,20503.140,20503.14,20503.140,20503.140,20503.140,20503.140,13430.2945,20503.14,20503.140,20503.140,20503.140,20503.140,20503.14,20503.140,20503.140,20503.140,20503.140,14.0,20503.14,19059.702143,295.01,5400.849921,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,6,5,3,4,20503.14
13349736,19746.0,854.077007,6619.089476,1.88,18.04,18.64,19

In [43]:
train_copy = df_training.copy()

In [44]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
train_copy.dropna(axis=0, thresh=152, inplace=True)
display(train_copy.head(5))
print(train_copy.shape)
#(195254, 154)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.640,20.566,23.050,25.3025,36.520,43.0475,51.344,61.170,69.73,89.398,109.920,147.578,189.372,235.8075,303.324,398.880,691.273,2687.6785,183608.57,21.0,19.247619,1.405762e+01,15.59,15.7700,15.770,15.770,15.770,15.7700,15.77,15.7700,15.77,15.770,15.77,15.7700,15.770,15.7700,15.94,15.9400,15.940,18.4400,18.440,18.4400,80.46,2.0,21.49,2.206173,19.93,20.086,20.242,20.398,20.554,20.710,20.866,21.022,21.178,21.334,21.49,21.646,21.802,21.958,22.114,22.270,22.426,22.582,22.738,22.894,23.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2017,10,2,4,2,15.94
13906939,229.0,11936.264192,91332.658153,15.59,16.218,18.440,18.840,19.770,22.8000,23.150,23.5500,33.368,38.938,48.20,64.560,81.470,96.712,134.004,173.4000,261.672,393.024,844.064,1813.2580,818332.70,3.0,16.490000,0.000000e+00,16.49,16.4900,16.490,16.490,16.490,16.4900,16.49,16.4900,16.49,16.490,16.49,16.4900,16.490,16.4900,16.49,16.4900,16.490,16.4900,16.490,16.4900,16.49,2.0,16.11,0.000000,16.11,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.11,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019,6,3,3,2,16.49
10522377,4.0,106.580000,65.938998,17.44,30.574,43.708,56.842,69.976,83.1100,96.244,106.2400,109.960,113.680,117.40,12

(195254, 154)


In [66]:
train_copy[train_copy['promoter_std'].isna()]['promoter_std'] = 0
train_copy[train_copy['promoter_std'].isna()]

C:\Users\sgopalakrish\AppData\Local\Temp\ipykernel_472\2575313566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_copy[train_copy['promoter_std'].isna()]['promoter_std'] = 0


,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10522377,4.0,106.580000,65.938998,17.44,30.574,43.708,56.8420,69.976,83.1100,96.244,106.240,109.960,113.680,117.400,121.120,124.840,128.5600,134.228,140.8700,147.512,154.1540,160.796,167.4380,174.08,18.0,129.800000,2.924570e-14,129.80,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.80,1.0,129.80,NaN,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2014,3,4,2,4,129.80
13346721,1877.0,1814.360027,22057.073808,1.10,16.110,18.440,18.8400,22.800,23.5500,34.650,37.680,46.100,55.824,67.650,78.232,94.600,118.7160,145.920,190.3200,252.208,344.1600,582.414,1625.7060,689747.70,24.0,247.489583,2.717779e+02,24.10,27.550,53.751,69.270,69.510,69.670,89.947,121.223,141.688,173.920,190.780,203.8505,223.466,230.3525,231.915,244.6500,248.318,283.2505,595.611,959.3445,1001.73,1.0,69.67,NaN,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,10,4,4,3,70.55
10524816,720.0,368.485708,2930.588173,11.02,18.040,18.440,22.7625,23.100,36.0800,41.694,46.100,55

In [60]:
fsddf = train_copy[train_copy['band_std'].isna()]
fsddf['band_std'] = 0
fsddf

C:\Users\sgopalakrish\AppData\Local\Temp\ipykernel_472\1682541956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fsddf['band_std'] = 0


,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10740322,255.0,743.540745,3109.405009,14.91,15.770,18.240,18.725,21.590,22.8000,23.050,23.482,31.370,37.200,46.100,56.937,86.424,118.710,151.954,193.6300,281.618,397.188,975.022,4342.890,31576.35,1.0,23.05,0,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,23.05,1.0,23.050,NaN,23.05,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.05,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.050,23.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2016,12,5,1,2,18.44
11741701,173.0,1167.616127,6976.247576,11.02,15.770,16.110,16.780,18.640,18.8400,22.544,23.100,26.326,35.032,46.600,70.980,91.200,114.636,126.100,161.3600,265.500,445.752,787.752,1802.146,70048.16,1.0,52306.18,0,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,52306.18,4.0,241170.810,270939.496916,62.74,2016.574,3970.408,5924.242,7878.076,9831.910,11785.744,36222.190,105623.860,175025.530,244427.20,313828.870,383230.540,452632.210,475766.100,475766.100,475766.100,475766.100,475766.100,475766.100,475766.10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2018,8,3,3,4,52431.28
13699205,1347.0,1708.474439,26644.983853,11.02,18.240,18.6

In [65]:
fsddf3 = train_copy[train_copy['promoter_std'].isna()]
fsddf3

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10522377,4.0,106.580000,65.938998,17.44,30.574,43.708,56.8420,69.976,83.1100,96.244,106.240,109.960,113.680,117.400,121.120,124.840,128.5600,134.228,140.8700,147.512,154.1540,160.796,167.4380,174.08,18.0,129.800000,2.924570e-14,129.80,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.800,129.8000,129.80,1.0,129.80,NaN,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,129.80,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2014,3,4,2,4,129.80
13346721,1877.0,1814.360027,22057.073808,1.10,16.110,18.440,18.8400,22.800,23.5500,34.650,37.680,46.100,55.824,67.650,78.232,94.600,118.7160,145.920,190.3200,252.208,344.1600,582.414,1625.7060,689747.70,24.0,247.489583,2.717779e+02,24.10,27.550,53.751,69.270,69.510,69.670,89.947,121.223,141.688,173.920,190.780,203.8505,223.466,230.3525,231.915,244.6500,248.318,283.2505,595.611,959.3445,1001.73,1.0,69.67,NaN,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,69.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,10,4,4,3,70.55
10524816,720.0,368.485708,2930.588173,11.02,18.040,18.440,22.7625,23.100,36.0800,41.694,46.100,55

In [17]:
df_training.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip',compression='gzip')

In [20]:
df_testing.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip',compression='gzip')